<a href="https://colab.research.google.com/github/wjunwei2001/TailorMade/blob/main/tiktok_llm_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate
# !pip install InstructorEmbedding
!pip install -U sentence-transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import re
import spacy
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import torch
import sentence_transformers
import accelerate
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.losses import ContrastiveTensionDataLoader
from datasets import Dataset

In [3]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('intfloat/e5-large-v2')
# input_texts = [
#     'query: how much protein should a female eat',
#     'query: summit define',
#     "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#     "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."
# ]
# embeddings = model.encode(input_texts, normalize_embeddings=True)

In [4]:
# Import SOTA text embedding model bsaed on the universally recognised Massive Text Embedding Benchmark (MTEB)

# model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer('EZlee/e-commerce-bert-base-multilingual-cased')
# model = SentenceTransformer("Salesforce/SFR-Embedding-2_R")
# model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
# dimensions = 512
# model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)
# model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)
# model = SentenceTransformer("avsolatorio/GIST-Embedding-v0", revision=None)
model = SentenceTransformer('intfloat/e5-base-v2') # One of the top model for retrieval with instruction task

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

## LLM Finetuning (Self supervised)

In [5]:
df = pd.read_csv('amazon_data.csv')

<ipython-input-5-ef29398646c6>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('amazon_data.csv')


In [7]:
df = df.drop_duplicates(subset='title') # remove all duplicate products

In [8]:
df['description'].replace(np.nan, '', inplace=True)
# df['description'] = df['description'].apply(''.join)

In [9]:
df.head(5)

,title,original_price,sale_price,discount,rating,review_count,main_category,sub_category_1,sub_category_2,rankings,description,insights,purchase_cnt_prev_month,store_name,supplier,is_available,url
0,cycling sunglasses polarized men women sport s...,NaN,$7.99,NaN,3.3,3,Sports and Outdoors,Sports & Outdoors,Cycling Glasses & Goggles,"#52,093 in Sports & Outdoors (See Top 100 in S...",['【Flexible and Frame-less Design Frame】: The ...,NaN,NaN,ZJMZZM,ZJMZZM,In Stock,/sspa/click?ie=UTF8&spc=MToyMTgzOTMwMDMwOTIxMD...
1,"polarized sport sunglasses for men women, idea...",NaN,$19.99,NaN,4.5,38,Sports and Outdoors,Sports & Outdoors,Cycling Glasses & Goggles,"#270,555 in Sports & Outdoors (See Top 100 in ...",,NaN,NaN,ACBLUCE Store,ACBLUCE,In Stock,/sspa/click?ie=UTF8&spc=MToyMTgzOTMwMDMwOTIxMD...
2,"professional cycling glasses for men women, sp...",NaN,$8.49,NaN,4.6,54,Sports and Outdoors,Sports & Outdoors,Cycling Glasses & Goggles,"#2,851 in Sports & Outdoors (See Top 100 in Sp...",,"['Appearance_POSITIVE', 'Value_POSITIVE', 'Dar...",1K+ bought in past month,ROIBEAO,Roibeao Direct,In Stock,/sspa/click?ie=UTF8&spc=MToyMTgzOTMwMDMwOTIxMD...
3,rockbros bike bicycle cargo rack rear bike rac...,$43.99,$43.99,NaN,4.3,"2,803",Sports and Outdoors,Sports & Outdoors,Bike Cargo Racks,"#11,213 in Sports & Outdoors (See Top 100 in S...",['QUICK RELEASE CLAMP for easily mount on the ...,"['Quality_POSITIVE', 'Performance_POSITIVE', '...",700+ bought in past month,ROCKBROS Store,ROCKBROS LLC,In Stock,/RockBros-Bicycle-110-165-Capacity-Mountain/dp...
4,boao 20 pieces presta valve core bike replacem...,NaN,$9.99,NaN,4.6,832,Sports and Outdoors,Sports & Outdoors,Bike Tubes,"#9,574 in Sports & Outdoors (See Top 100 in Sp...",['Package include: 20 pieces presta valve core...,"['Quality_POSITIVE', 'Performance_POSITIVE', '...",600+ bought in past month,Boao Store,Hiroda US,In Stock,/Pieces-Replacement-Tubeless-Vittoria-Continen...


In [10]:
df['main_category'] = df['main_category'].str.lower()
df['main_category'].value_counts()

main_category
computers                   4363
home and kitchen            4230
baby products               4208
electronics                 3854
boy's fashion               3619
mens's fashion              3521
girl's fashion              3119
sports and outdoors         2853
beauty and personal care    2690
toys and games              2415
women's fashion             2016
main_category                  1
Name: count, dtype: int64

In [11]:
df['description'][0]

"['【Flexible and Frame-less Design Frame】: The newest Tr90 frame-less design for cycling glasses which make cycling sunglasses becoming more stylish and lightweight. The glasses leg is made of rubber which is flexible and comfortable.', '【UV400 Polarized Lenses】These Polarized Sport Sunglasses are took premium polarized lenses which are 100% UVA and UVB filtering, reduce reflections and glare for a clearer view. So your child can perform to the best of their ability in the Contest.', '【Ventilation Structure】: Our Sports Sunglasses have an excellent ventilation system with vents on the lenses and legs. This ventilation system not only prevents the lenses from fogging up, but also makes them more breathable when worn.', '【Designed for Sports】The Youth Sport Sunglasses are ideal for outdoor sports, such as baseball, softball, climbing, driving, fishing, golf, hiking, running, skiing, cycling and so on.', '【Anti Slip Design】: The legs of the polarized sunglasses are made of non-slip rubber

In [12]:
df['ecommerce_text'] = df['title'] + ' ' + df['description']

In [13]:
# Convert text to lowercase
df['ecommerce_text'] = df['ecommerce_text'].str.lower()

# Remove special characters and numbers
df['ecommerce_text'] = df['ecommerce_text'].apply(lambda x: re.sub('[^a-z\s]', '', str(x)))

In [14]:
df['ecommerce_text'][5]

'air pump replacement nozzles  pack inflatable pump nozzles head air pump inflator adaptor accessory for partial air bed air mattress etc  sizes it respectively in small medium and large size mouth easily meeting your needs to inflatable things of different mouth sizes made of quality plastic durable for long time using and lightweight for carrying in your travel bag and family toolbox choose the nozzle of proper size accordingly and install it on the air pump and then inflate the target object the small size one fit for small toys medium and large one fits for swimming pool boat inflatable bed this nozzles head is suitable for different kinds of pumps helping you to inflate swimming rings air bed air mattress inflatable pool sofa and more a good tool accessory for outdoor activities'

In [15]:
new_df = df.groupby('main_category').apply(lambda x: x.sample(min(len(x), 1000))).reset_index(drop=True)

# Verify the counts in the new DataFrame
print(new_df['main_category'].value_counts())

main_category
baby products               1000
beauty and personal care    1000
boy's fashion               1000
computers                   1000
electronics                 1000
girl's fashion              1000
home and kitchen            1000
mens's fashion              1000
toys and games              1000
sports and outdoors         1000
women's fashion             1000
main_category                  1
Name: count, dtype: int64


In [16]:
train_examples = new_df['ecommerce_text'].tolist()

In [17]:
train_dataloader = ContrastiveTensionDataLoader(train_examples, batch_size=8, pos_neg_ratio=8)
train_loss = losses.ContrastiveTensionLoss(model=model)

model.fit(
    [(train_dataloader, train_loss)],
    epochs=5,
)

Step,Training Loss
500,7.905200
1000,4.380300
1500,3.625300
2000,3.685300
2500,3.687800
3000,3.602000
3500,3.651200


In [18]:
!pip install huggingface_hub

In [19]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [21]:
model.push_to_hub("ecommerce_text_embedding_retrieval_v2")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/wjunwei/ecommerce_text_embedding_retrieval_v2/commit/efd0a1a71adf6896ec9d47cb0662dc6aeeddc68f'

In [41]:
ecommerce_model = SentenceTransformer("wjunwei/ecommerce_text_embedding_retrieval_v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Using the model for generating similarity score

In [23]:
# Do the same text processing for inference
df['title'] = df['title'].str.lower()
df['title'] = df['title'].apply(lambda x: re.sub('[^a-z\s]', '', str(x)))

df['product_embedding'] = 'passage: ' + df['ecommerce_text']
df['product_embedding'] = df['product_embedding'].apply(lambda x: ecommerce_model.encode(x))

In [24]:
df['ecommerce_text'][105]

'skull cap helmet liner beanie cooling mesh cycling running hat for men women fits under helmets black  black '

In [26]:
recipient_and_style_prompt = "query: Does this product meet the needs of a middle-aged man who is phiscally active?"

# user_input = "my uncle who loves to exercise"
# user_input2 = "my middle age wife"

In [27]:
user_input_embedding = ecommerce_model.encode(recipient_and_style_prompt)
# user_input_embedding2 = ecommerce_model.encode(recipient_and_style_prompt, normalize_embeddings=True)

In [28]:
def calculate_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])

# result1 = calculate_similarity(user_input_embedding, df['product_embedding'][105])
# # result2 = calculate_similarity(user_input_embedding2, df['product_embedding'][105])

# print(result1)
# # print(result2)

In [79]:
# similarities = []

# for product_embedding in df['product_embedding']:
#     similarity = calculate_similarity(user_input_embedding, product_embedding)
#     similarities.append(similarity)

# # Add similarity results to the DataFrame
# df['similarity_user'] = similarities

# # Rank products based on similarity for each user input
# df_sorted_user = df.sort_values(by='similarity_user', ascending=False)

# # Display top few products for each user input
# top_n = 10
# print(f"Top {top_n} products:")
# print(df_sorted_user[['title', 'similarity_user']].head(top_n))

## Validation against Keyword Matching Algorithm

In [68]:
# Example queries
queries = [
    "query: Does this product meet the needs of a fitness enthusiast who is looking for durable products?",
    "query: Does this product meet the needs of a gamer who is looking for high-tech products?",
    "query: Does this product meet the needs of a traveler who is looking for portable products?",
    "query: Does this product meet the needs of an office worker who is looking for ergonomic products?",
    "query: Does this product meet the needs of a student who is looking for budget-friendly products?",
    "query: Does this product meet the needs of a parent who is looking for educational products?",
    "query: Does this product meet the needs of a photographer who is looking for high-quality products?",
    "query: Does this product meet the needs of a cook who is looking for durable kitchen products?",
    "query: Does this product meet the needs of a music lover who is looking for premium sound products?",
    "query: Does this product meet the needs of an outdoor adventurer who is looking for rugged products?",
    "query: Does this product meet the needs of a fashion-forward person who is looking for trendy products?",
    "query: Does this product meet the needs of a DIY enthusiast who is looking for reliable tools?",
    "query: Does this product meet the needs of a health-conscious person who is looking for organic products?",
    "query: Does this product meet the needs of a pet owner who is looking for pet-friendly products?",
    "query: Does this product meet the needs of a commuter who is looking for compact products?"
]

# Example products
products = [
    "passage: stylish red running shoes, perfect for long-distance runs with breathable mesh and cushioned sole for maximum comfort",
    "passage: comfortable wireless headphones with noise cancellation, ideal for travel and work, featuring a long battery life and high-fidelity sound",
    "passage: red sneakers, combining fashion and functionality, suitable for casual wear and light athletic activities",
    "passage: bluetooth wireless earphones, compact and lightweight, offering high-quality sound and convenient hands-free calling",
    "passage: black leather biker jacket, made from premium leather, designed for a rugged and stylish look",
    "passage: slim-fit black leather jacket, featuring a sleek design, suitable for both casual and formal occasions",
    "passage: BPA-free sports water bottle, eco-friendly and durable, perfect for staying hydrated during workouts",
    "passage: insulated sports water bottle, keeping drinks cold for up to 24 hours, ideal for outdoor activities and gym sessions",
    "passage: tempered glass smartphone screen protector, offering high clarity and robust protection against scratches and drops",
    "passage: scratch-resistant smartphone screen protector, providing long-lasting protection with easy installation",
    "passage: slim-fit blue denim jeans, versatile and stylish, suitable for a variety of occasions from casual outings to semi-formal events",
    "passage: relaxed-fit blue denim jeans, offering comfort and a laid-back style, perfect for everyday wear",
    "passage: non-slip yoga mat, ensuring stability during yoga practices, made from eco-friendly materials",
    "passage: eco-friendly yoga mat, providing excellent grip and cushioning, made from sustainable materials",
    "passage: high-performance gaming laptop, equipped with a powerful processor and graphics card, ideal for immersive gaming experiences",
    "passage: lightweight gaming laptop, combining portability with high performance, perfect for gaming on the go",
    "passage: waterproof digital wristwatch, featuring a sleek design and multiple functionalities, ideal for outdoor activities",
    "passage: smart digital wristwatch, with fitness tracking, notifications, and customizable watch faces, perfect for tech enthusiasts",
    "passage: rechargeable electric toothbrush, ensuring thorough cleaning with multiple brushing modes and a long-lasting battery",
    "passage: portable electric toothbrush, compact and travel-friendly, featuring a powerful motor for effective cleaning",
    "passage: durable resistance bands, ideal for fitness enthusiasts, providing various levels of resistance for different exercises",
    "passage: high-tech gaming mouse with customizable buttons and RGB lighting, perfect for gamers seeking precision",
    "passage: portable power bank with fast charging capabilities, essential for travelers to keep their devices charged on the go",
    "passage: ergonomic office chair with lumbar support, designed to provide comfort during long working hours",
    "passage: budget-friendly laptop with essential features, ideal for students for their daily academic tasks",
    "passage: educational tablet for kids, featuring interactive learning apps and parental controls",
    "passage: high-quality DSLR camera with advanced features, perfect for photographers looking to capture stunning images",
    "passage: durable non-stick frying pan, ideal for cooks who need reliable kitchen tools",
    "passage: premium over-ear headphones with high-fidelity sound, perfect for music lovers",
    "passage: rugged hiking boots with waterproof features, designed for outdoor adventurers",
    "passage: trendy sunglasses with UV protection, perfect for fashion-forward individuals",
    "passage: reliable cordless drill with multiple speed settings, essential for DIY enthusiasts",
    "passage: organic skincare set with natural ingredients, ideal for health-conscious individuals",
    "passage: pet-friendly vacuum cleaner with HEPA filter, perfect for pet owners to keep their homes clean",
    "passage: compact foldable electric scooter, perfect for commuters looking for a convenient transportation option"
]

# Relevant items labelled by existing LLM (GPT4)
relevant_items = [
    [0, 1, 6, 20], [1, 14, 21], [1, 3, 22], [1, 23, 24], [0, 3, 25],
    [2, 18, 26], [1, 7, 27], [1, 8, 28], [1, 9, 29], [2, 6, 30],
    [0, 11, 31], [0, 12, 32], [3, 13, 33], [1, 14, 34], [2, 15, 35]
]


In [69]:
def keyword_overlap(query, product):
    query_tokens = set(query.lower().split())
    product_tokens = set(product.lower().split())
    return len(query_tokens & product_tokens) / len(query_tokens.union(product_tokens))

In [70]:
ranked_products_finetuned_all = []
ranked_products_keyword_all = []

for query in queries:
    user_input_embedding = ecommerce_model.encode(query)

    similarities_finetuned = []
    similarities_keyword = []

    for product in products:
        product_embedding = ecommerce_model.encode(product)

        similarity_finetuned = calculate_similarity(user_input_embedding, product_embedding)
        similarity_keyword = keyword_overlap(query, product)

        similarities_finetuned.append(similarity_finetuned)
        similarities_keyword.append(similarity_keyword)

    # Sort products based on similarities
    df = pd.DataFrame({
        'product': products,
        'similarity_finetuned': similarities_finetuned,
        'similarity_keyword': similarities_keyword
    })

    df_sorted_finetuned = df.sort_values(by='similarity_finetuned', ascending=False)
    df_sorted_keyword = df.sort_values(by='similarity_keyword', ascending=False)

    ranked_products_finetuned_all.append(df_sorted_finetuned.index.tolist())
    ranked_products_keyword_all.append(df_sorted_keyword.index.tolist())

In [71]:
def recall_at_k(ranked_list, relevant_items, k):
    return np.mean([len(set(ranked_list[i][:k]) & set(relevant_items[i])) / len(relevant_items[i]) for i in range(len(ranked_list))])

In [77]:
k = 20
recall_finetuned = recall_at_k(ranked_products_finetuned_all, relevant_items, k)
recall_keyword = recall_at_k(ranked_products_keyword_all, relevant_items, k)

print(f"Fine-tuned Model Recall@{k}: {recall_finetuned}")
print(f"Keyword Matching Recall@{k}: {recall_keyword}")

Fine-tuned Model Recall@20: 0.611111111111111
Keyword Matching Recall@20: 0.5666666666666668
